In [4]:
from tkinter import *
from tkinter import ttk

root = Tk()
root.geometry('300x200')
root.title('バーチャルトレーディングシステム')

frame = ttk.Frame(root)
frame.pack(fill = BOTH, pady=20)

frame_top = ttk.Frame(frame)
frame_top.pack(expand = True)

frame_bottom = ttk.Frame(frame)
frame_bottom.pack(expand = True)

label = ttk.Label(frame_top, text='ユーザー名')
label.pack(side = LEFT)

entry = ttk.Entry(frame_top)
entry.pack(side = RIGHT)

label2 = ttk.Label(frame_bottom, text='パスワード')
label2.pack(side = LEFT)

entry2 = ttk.Entry(frame_bottom)
entry2.pack(side = RIGHT)

link = ttk.Label(frame, text='パスワードを忘れた場合はこちら',foreground='blue',cursor='hand1',font=('MSゴシック', 10, 'underline'))
link.pack()

root.mainloop()

: 